In [1]:
from CS2_bo_functions import LHS_Design
from CS2_bo_functions import create_y_data
from CS2_bo_functions import calc_y_exp
from CS2_bo_functions import test_train_split
from CS2_bo_functions import create_sse_data
from scipy.stats import qmc
import numpy as np
import pandas as pd
import os

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
#LHS Design in python - May switch code to use this instead for conciseness
emulator = True
seed = 9
LHS_Points =20
d = 4
sampler = qmc.LatinHypercube(d=d, seed = seed)
sample = sampler.random(n=LHS_Points)
l_bounds =  [-2, -2, -10 ,-2]
u_bounds =  [0, 0 , 0, 2 ]
LHS = qmc.scale(sample, l_bounds, u_bounds)
# print(LHS)

[[-1.68702492e+00 -8.28681721e-01 -4.30157408e+00  8.44493183e-01]
 [-9.71607463e-01 -1.99153801e+00 -6.93019682e+00 -9.83647526e-01]
 [-2.65877345e-03 -1.44372480e+00 -1.74247216e+00 -4.13030839e-01]
 [-6.00562605e-01 -1.83062144e-01 -3.49165112e+00  1.24307803e+00]
 [-1.93156035e+00 -1.77053337e+00 -5.14959054e+00  1.45185030e+00]
 [-1.12797472e+00 -1.87825909e+00 -8.99387037e+00 -7.97247710e-01]
 [-4.88286257e-01 -3.91281005e-01 -7.85410346e+00  4.89100625e-01]
 [-1.79232382e+00 -2.08973078e-01 -5.68506577e+00  7.88122113e-02]
 [-5.47772990e-01 -1.07928579e+00 -8.09754320e+00  3.87549301e-01]
 [-1.01265184e+00 -6.96278158e-01 -1.06638727e+00 -1.64521967e+00]
 [-2.37705637e-01 -1.16234216e+00 -9.39104807e+00  7.84270221e-01]
 [-1.49312375e+00 -4.76279172e-01 -1.60482059e-01 -1.83786640e+00]
 [-1.55959529e+00 -1.65946892e+00 -2.63373418e+00  1.76612144e+00]
 [-1.37664673e+00 -9.67267525e-01 -6.40012468e+00  1.15608466e+00]
 [-8.72320853e-01 -1.30746047e+00 -5.03668997e-01 -1.43588691e

In [5]:
#Creates Experimental data and saves to a csv
Constants = np.array([[-200,-100,-170,15],
                      [-1,-1,-6.5,0.7],
                      [0,0,11,0.6],
                      [-10,-10,-6.5,0.7],
                      [1,0,-0.5,-1],
                      [0,0.5,1.5,1]])

Xexp = np.array([[-0.558,-0.050,0.623,-0.82, 0.22],
                 [1.442,0.467,0.028,0.62,0.30]])

n = Xexp.shape[1]
noise_std = 0.01
Yexp = calc_y_exp(Constants, Xexp, noise_std)

df = pd.DataFrame({'X1exp': Xexp[0, :], 'X2exp': Xexp[1, :], 'Yexp':Yexp[:]})

path = 'Input_CSVs/Exp_Data'
if not os.path.exists(path):
    os.makedirs(path)
df.to_csv(os.path.join(path,'n='+str(n)+'.csv'))

In [36]:
#Array for Standard Approach
all_p_standard = pd.DataFrame(LHS, columns = ["a1", "a2", "a3", "a4"])


#Array for Emulator approach
df = all_p_standard.iloc[np.repeat(np.arange(len(all_p_standard)), n)]
df = df.reset_index(drop = True)
Xexp_repeat = np.repeat(Xexp, LHS_Points, axis = 0).reshape(Xexp.shape[0],-1)
# print(Xexp_repeat)
#Add Xexp 
for i in range(Xexp.shape[0]):
    df['X'+str(i+1)+'exp'] = Xexp_repeat[i]
all_p_emulator = df
print(all_p_emulator)

          a1        a2        a3        a4  X1exp  X2exp
0  -1.687025 -0.828682 -4.301574  0.844493 -0.558  1.442
1  -1.687025 -0.828682 -4.301574  0.844493 -0.050  0.467
2  -1.687025 -0.828682 -4.301574  0.844493  0.623  0.028
3  -1.687025 -0.828682 -4.301574  0.844493 -0.820  0.620
4  -1.687025 -0.828682 -4.301574  0.844493  0.220  0.300
..       ...       ...       ...       ...    ...    ...
95 -0.179497 -0.562315 -3.755157 -0.111309 -0.558  1.442
96 -0.179497 -0.562315 -3.755157 -0.111309 -0.050  0.467
97 -0.179497 -0.562315 -3.755157 -0.111309  0.623  0.028
98 -0.179497 -0.562315 -3.755157 -0.111309 -0.820  0.620
99 -0.179497 -0.562315 -3.755157 -0.111309  0.220  0.300

[100 rows x 6 columns]


In [5]:
#Create Training Data and Testing Data
#Set noise parameters and load csv file to generate training data
# all_p = LHS_Design("LHS_Toy_3_Input.csv")
t = str(len(all_p))

#Create y or SSE data
#Save data
if emulator == False:
    all_2_in_y = create_sse_data(q,all_p,Xexp,Yexp)
    all_2_in_y_ln_obj = create_sse_data(q,all_p,Xexp,Yexp, obj = "LN_obj")
    all_2_data = np.column_stack((all_p, all_2_in_y))
    all_2_data_ln_obj = np.column_stack((all_p, all_2_in_y_ln_obj))
    titles_all = np.array(['all_2_data','all_2_ln_obj_data'])
    all_array = np.array([all_2_data,all_2_data_ln_obj])

else:
    all_3_in_y = create_y_data(all_p)
    all_3_data = np.column_stack((all_p, all_3_in_y))
    titles_all = np.array(['all_3_data'])
    all_array = np.array([all_3_data])

#Save Train Data/Test Data
for i in range(len(titles_all)):
    df_all = pd.DataFrame(all_array[i], columns = ["Theta_1","Theta_2","X Coord","y-value"])
    path = 'Input_CSVs/Train_Data/'+titles_all[i]+"/"
    if not os.path.exists(path):
        os.makedirs(path)
    df_all.to_csv(os.path.join(path,'t='+t)+'.csv')

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 40 and the array at index 1 has size 3